# Sprint 深層学習スクラッチ　リカレントニューラルネットワーク

## 1.このSprintについて

**<u>Sprintの目的</u>**

- スクラッチを通してリカレントニューラルネットワークの基礎を理解する

**<u>どのように学ぶか</u>**

スクラッチでリカレントニューラルネットワークの実装を行います。

## 2.リカレントニューラルネットワークスクラッチ

**リカレントニューラルネットワーク（RNN）** のクラスをスクラッチで作成していきます。NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。


フォワードプロパゲーションの実装を必須課題とし、バックプロパゲーションの実装はアドバンス課題とします。


クラスの名前はScratchSimpleRNNClassifierとしてください。クラスの構造などは以前のSprintで作成したScratchDeepNeuralNetrowkClassifierを参考にしてください。

## 【問題1】SimpleRNNのフォワードプロパゲーション実装
SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。


フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。


バッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

In [1]:
import numpy as np

In [34]:
class SimpleRNN:
    """
    SimpleRNNのフォワードプロパゲーション実装
    Parameters
    ----------
    n_nodes : RNNのノード数 (問題2ではw_x.shape[1] # 4)
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes, initializer, optimizer):
        self.n_nodes = n_nodes
        self.optimizer = optimizer
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.w_x = initializer.W(n_features, n_nodes) # 入力に対する重み
        self.w_h = initializer.W(n_nodes, n_nodes)  # 時刻t-1の状態に対する重み
        self.b = initializer.B(n_nodes) # バイアス項

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_sequences, n_features)
            入力
        Returns
        ----------
        : 次の形のndarray, shape (batch_size, n_nodes)
            出力
        """
        self.X = X
        batch_size, n_sequences, n_features = X.shape
       
        # 出力用の入れ物を用意
        a = h = np.zeros((batch_size, n_sequences, n_nodes))

        # 問題1数式通りに実装
        for t in range(n_sequences):  # 系列数n_sequences回繰り返す
          h[:, t-1] = np.tanh(a[:, t-1]) # 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_sequences, n_nodes)
          # print('h[:, t-1]', h)
          a[:, t] = X[:, t]@self.w_x + h[:, t-1]@self.w_h + self.b # 時刻tの活性化関数を通す前の状態 (batch_size, n_sequences, n_nodes)
          # print('a[:, t]', a)
        
        # 最後の出力時の状態 (batch_size, n_nodes)
        h[:, -1] = np.tanh(a[:, -1])
        # print('h[:, -1]', h)

        return h[:, -1]

**初期化**

In [17]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma

    def W(self, *shape):
        """
        重みの初期化
        """
        # 各種shapeに対応できるよう改造(n_nodes1, n_nodes2)⇒(*shape)
        W = self.sigma * np.random.randn(*shape)
        return W

    def B(self, *shape):
        """
        バイアスの初期化
        """
        # 各種shapeに対応できるよう改造(n_nodes1, n_nodes2)⇒(*shape)
        B = self.sigma * np.random.randn(*shape)
        return B

## 【問題2】小さな配列でのフォワードプロパゲーションの実験
小さな配列でフォワードプロパゲーションを考えてみます。


入力x、初期状態h、重みw_xとw_h、バイアスbを次のようにします。


ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [21]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100 # (batch_size, n_sequences, n_features)
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100 # (n_features, n_nodes)
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100 # (n_nodes, n_nodes)
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes)) # (batch_size, n_nodes)
b = np.array([1, 1, 1, 1]) # (n_nodes,)

In [35]:
# インスタンス化
rnn = SimpleRNN(n_nodes=n_nodes, initializer=SimpleInitializer(0.01), optimizer=None)
rnn.w_x, rnn.w_h, rnn.b = w_x, w_h, b

In [36]:
# フォワードプロパゲーション
rnn.forward(x)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])